In [3]:
import numpy as np
from multiple_factor import DGP2, Inferece2
from dgp import DGP

def reject_prob(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    phi_tau = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        inf = Inferece2(Y, D, tuple_idx, design)
        phi_tau[i] = inf.phi_tau
    return np.mean(phi_tau)


def risk(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    mse = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        ate = np.mean(Y[D[:,0]==1]) - np.mean(Y[D[:,0]==0])
        mse[i] = (ate - tau)**2
    return np.mean(mse)


def test(q,k):
    print(q,k)
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='S4'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='C'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='RE'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MP-B'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MT'))
    

def test_infer(q,k):
    print(q,k)
    #print("null")
    print(reject_prob(q, k, 1280, tau=0, ntrials=500, more=False, design='S4'))
    print(reject_prob(q, k, 1280, tau=0, ntrials=500, more=False, design='C'))
    print(reject_prob(q, k, 1280, tau=0, ntrials=500, more=True, design='MT'))
    print(reject_prob(q, k, 1280, tau=0, ntrials=500, more=False, design='MT'))
    #print("alternative")
    #print(reject_prob(q, k, 1280, tau=0.05, ntrials=500, more=False, design='S4'))
    #print(reject_prob(q, k, 1280, tau=0.05, ntrials=500, more=False, design='C'))
    #print(reject_prob(q, k, 1280, tau=0.05, ntrials=500, more=True, design='MT'))
    #print(reject_prob(q, k, 1280, tau=0.05, ntrials=500, more=False, design='MT'))


#[test(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
#test(1,1)
#test(10,1)
#test(1,2)
#test(2,2)
#test(10,6)

#test_infer(1,1)
#test_infer(10,1)
#test_infer(1,2)
#test_infer(2,2)
#test_infer(10,6)
#print(reject_prob(10, 4, 1280, tau=0, ntrials=500, more=False, design='MT') )
#print(reject_prob(10, 4, 1280, tau=0.05, ntrials=500, more=False, design='MT') )

designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results = []

for m in designs:
    print(m)
    if m == 'MT2':
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT') 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    else:
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    results.append(result)
    baseline = results[0][(1,1)]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]/baseline))

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results_mse = []

num_cores = multiprocessing.cpu_count()
for m in designs:
    print(m)
    qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
    def processInput(qk):
        q, k = qk
        if m == 'MT2':
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT')
        else:
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
        return (q,k,mse)
    ret = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
    result = {(re[0], re[1]):re[2] for re in ret}
    results_mse.append(result)
    baseline = results_mse[0][(1,1)]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]/baseline))

MT
1.000 & 1.057 & 1.093 & 1.004 & 1.137 & 1.103 \\
0.768 & 0.770 & 0.772 & 0.774 & 0.776 & 0.792 \\
0.403 & 0.413 & 0.456 & 0.478 & 0.536 & 0.666 \\
0.225 & 0.307 & 0.413 & 0.580 & 0.715 & 0.857 \\
0.267 & 0.414 & 0.497 & 0.649 & 0.880 & 0.944 \\
C
1.359 & 1.452 & 1.440 & 1.274 & 1.365 & 1.454 \\
1.451 & 1.393 & 1.384 & 1.499 & 1.482 & 1.397 \\
1.358 & 1.441 & 1.492 & 1.391 & 1.421 & 1.401 \\
1.429 & 1.313 & 1.366 & 1.365 & 1.400 & 1.324 \\
1.422 & 1.426 & 1.416 & 1.408 & 1.478 & 1.514 \\
MT2
1.114 & 0.987 & 1.053 & 1.013 & 1.030 & 1.105 \\
0.754 & 0.798 & 0.712 & 0.732 & 0.779 & 0.901 \\
0.408 & 0.492 & 0.488 & 0.575 & 0.632 & 0.768 \\
0.300 & 0.392 & 0.560 & 0.680 & 0.798 & 1.078 \\
0.370 & 0.466 & 0.687 & 0.878 & 0.988 & 1.091 \\
S4
1.092 & 1.041 & 1.109 & 1.136 & 1.109 & 1.062 \\
1.151 & 1.107 & 1.053 & 1.131 & 1.017 & 1.077 \\
1.056 & 1.064 & 1.064 & 1.096 & 1.123 & 1.118 \\
1.163 & 1.038 & 1.146 & 1.066 & 1.040 & 1.098 \\
1.134 & 1.018 & 1.041 & 1.084 & 1.082 & 1.106 \\
MP-B
1.016 & 1.450 & 1.324 & 1.386 & 1.456 & 1.333 \\
0.769 & 1.426 & 1.354 & 1.356 & 1.378 & 1.458 \\
0.397 & 1.457 & 1.437 & 1.264 & 1.460 & 1.304 \\
0.234 & 1.451 & 1.354 & 1.318 & 1.423 & 1.411 \\
0.267 & 1.452 & 1.392 & 1.429 & 1.371 & 1.472 \\
RE
1.014 & 1.337 & 1.512 & 1.386 & 1.390 & 1.492 \\
0.808 & 1.489 & 1.299 & 1.480 & 1.432 & 1.419 \\
0.423 & 1.492 & 1.452 & 1.419 & 1.363 & 1.341 \\
0.292 & 1.427 & 1.360 & 1.320 & 1.479 & 1.533 \\
0.327 & 1.416 & 1.399 & 1.541 & 1.420 & 1.464 \\

In [4]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
designs = ['MT', 'MT2', 'C', 'S4']
results_null = []

num_cores = multiprocessing.cpu_count()
for m in designs:
    print(m)
    qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [1]]
    def processInput(qk):
        q, k = qk
        if m == 'MT2':
            prob = reject_prob(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT')
        else:
            prob = reject_prob(q, k, 1280, tau=0, ntrials=1000, more=False, design=m)
        return (q,k,prob)
    ret = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
    result = {(re[0], re[1]):re[2] for re in ret}
    results_null.append(result)
    for q in [1,2,4,8,10]:
        for k in [1]:
            if k<1:
                print("{:.3f} & ".format(result[(q,k)]), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]))

MT


MT
0.046 & 0.043 & 0.048 & 0.048 & 0.059 & 0.066 \\
0.035 & 0.039 & 0.045 & 0.046 & 0.046 & 0.040 \\
0.041 & 0.042 & 0.041 & 0.029 & 0.016 & 0.022 \\
0.026 & 0.018 & 0.014 & 0.012 & 0.017 & 0.020 \\
0.021 & 0.019 & 0.020 & 0.019 & 0.020 & 0.032 \\
MT2
0.045 & 0.057 & 0.044 & 0.046 & 0.043 & 0.056 \\
0.047 & 0.051 & 0.054 & 0.058 & 0.053 & 0.061 \\
0.060 & 0.049 & 0.044 & 0.045 & 0.049 & 0.045 \\
0.050 & 0.049 & 0.051 & 0.050 & 0.057 & 0.056 \\
0.053 & 0.062 & 0.045 & 0.053 & 0.058 & 0.057 \\
C
0.044 & 0.056 & 0.064 & 0.046 & 0.051 & 0.063 \\
0.062 & 0.057 & 0.053 & 0.037 & 0.054 & 0.073 \\
0.061 & 0.053 & 0.049 & 0.058 & 0.047 & 0.047 \\
0.051 & 0.049 & 0.043 & 0.048 & 0.061 & 0.052 \\
0.055 & 0.050 & 0.048 & 0.050 & 0.051 & 0.067 \\
S4
0.045 & 0.050 & 0.050 & 0.048 & 0.071 & 0.091 \\
0.043 & 0.055 & 0.046 & 0.058 & 0.065 & 0.093 \\
0.059 & 0.059 & 0.061 & 0.062 & 0.054 & 0.087 \\
0.055 & 0.054 & 0.065 & 0.064 & 0.068 & 0.073 \\
0.054 & 0.058 & 0.055 & 0.061 & 0.063 & 0.079 \\

In [2]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
designs = ['MT', 'MT2', 'C', 'S4']
results_null = []

num_cores = multiprocessing.cpu_count()
for m in designs:
    print(m)
    qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
    def processInput(qk):
        q, k = qk
        if m == 'MT2':
            prob = reject_prob(q, k, 1280, tau=0.05, ntrials=1000, more=True, design='MT')
        else:
            prob = reject_prob(q, k, 1280, tau=0.05, ntrials=1000, more=False, design=m) 
        return (q,k,prob)
    ret = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
    result = {(re[0], re[1]):re[2] for re in ret}
    results_null.append(result)
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]))

MT
0.449 & 0.441 & 0.417 & 0.438 & 0.439 & 0.449 \\
0.565 & 0.570 & 0.538 & 0.546 & 0.541 & 0.521 \\
0.827 & 0.791 & 0.744 & 0.675 & 0.570 & 0.473 \\
0.965 & 0.840 & 0.708 & 0.537 & 0.435 & 0.372 \\
0.892 & 0.728 & 0.602 & 0.485 & 0.373 & 0.338 \\
MT2
0.439 & 0.466 & 0.440 & 0.417 & 0.441 & 0.408 \\
0.571 & 0.574 & 0.578 & 0.570 & 0.541 & 0.524 \\
0.810 & 0.803 & 0.725 & 0.722 & 0.620 & 0.535 \\
0.912 & 0.827 & 0.728 & 0.603 & 0.490 & 0.442 \\
0.872 & 0.718 & 0.621 & 0.536 & 0.476 & 0.427 \\
C
0.344 & 0.351 & 0.384 & 0.372 & 0.380 & 0.365 \\
0.350 & 0.363 & 0.375 & 0.371 & 0.336 & 0.349 \\
0.347 & 0.354 & 0.352 & 0.342 & 0.369 & 0.367 \\
0.337 & 0.362 & 0.329 & 0.365 & 0.328 & 0.370 \\
0.309 & 0.382 & 0.340 & 0.354 & 0.331 & 0.376 \\
S4
0.433 & 0.415 & 0.457 & 0.432 & 0.471 & 0.517 \\
0.421 & 0.400 & 0.434 & 0.446 & 0.447 & 0.490 \\
0.439 & 0.446 & 0.412 & 0.441 & 0.479 & 0.502 \\
0.434 & 0.433 & 0.458 & 0.434 & 0.460 & 0.516 \\
0.435 & 0.484 & 0.443 & 0.455 & 0.478 & 0.489 \\
